In [1]:
import os
import re
import gymnasium as gym
import time
import copy
import random
import warnings
import numpy as np
import cv2
from time import sleep
sleep(0.0416)

import torch
import torchvision
import torch.optim as optim
import torch.nn as nn

from IPython import display
from skimage.color import rgb2gray
from skimage.transform import rescale
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
from collections import deque, namedtuple

In [2]:
def preprocess_frame(frame):
    # If frame is a tuple, extract the first element (assuming it's the image)
    if isinstance(frame, tuple):
        frame = frame[0]  # Adjust the index based on the structure
    
    # Convert to grayscale and rescale
    gray = rgb2gray(frame)
    scaled = rescale(gray, 0.5, anti_aliasing=True)
    return scaled


In [3]:
class DQN(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(DQN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(3456, 512),  # Adjust input size to match flattened convolutional output
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )


    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


In [4]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)
    
    def push(self, *args):
        self.memory.append(Transition(*args))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [5]:
# Hyperparameters
BATCH_SIZE = 32
GAMMA = 0.99
EPSILON = 0.05  # Exploration probability
TARGET_UPDATE = 10  # How often to update the target network
MEMORY_CAPACITY = 10000

# Setup the Atari environment
env = gym.make('Pong-v4', render_mode="human")
n_actions = env.action_space.n
print(env.unwrapped.get_action_meanings())

# Create networks
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_net = DQN((1, 84, 84), n_actions).to(device)
target_net = DQN((1, 84, 84), n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

# Replay memory
memory = ReplayMemory(MEMORY_CAPACITY)

# Optimizer
optimizer = optim.Adam(policy_net.parameters())

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [6]:
def select_action(state, steps_done):
    sample = random.random()
    eps_threshold = EPSILON
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    next_state_batch = torch.cat(batch.next_state)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = target_net(next_state_batch).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    loss = nn.SmoothL1Loss()(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [8]:
def reward_function(observation, action, next_observation):
    reward = 0

    # Example reward structure based on game events
    if next_observation['score'] > observation['score']:
        reward += 1  # Positive reward for increasing the score
    elif next_observation['lives'] < observation['lives']:
        reward -= 1  # Negative reward for losing a life
    elif action == "shoot" and next_observation['enemy_defeated']:
        reward += 5  # Reward for defeating an enemy
    
    return reward


In [9]:
num_episodes = 50
for i_episode in range(num_episodes):
    state = env.reset()
    state = preprocess_frame(state)
    state = torch.tensor(state, device=device, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    for t in range(10000):
        env.render()
        action = select_action(state, t)
        obs, reward, done, _, _= env.step(action.item())
        if reward == 0:
            reward = -0.1
        #cv2.imshow("Game", next_state)
        
        
        next_state = preprocess_frame(obs)
        next_state = torch.tensor(next_state, device=device, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        reward = torch.tensor([reward], device=device)


        memory.push(state, action, next_state, reward)

        state = next_state

        optimize_model()

        if done:
            break
    
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

env.close()

c:\Users\macenh\Documents\RCOS\ReinforcementLearning\MaxCut_RL\tutorials\holden_macentee\.venv\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi

hi



KeyboardInterrupt: 